In [1]:
import logging


logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s: %(message)s",
    force=True,
)

In [2]:
from src.datasets.dataset import CnnDataset
from src.datasets.mnist_dataset import MiniMNISTDataset
from src.models.cnn import CNNParams, ConvLayerParams, ConvParams
from src.models.compression.enums import Activation, NNParamsCompMode, QMode
from src.models.eval import NNArchitectureEvaluator
from src.models.mlp import FCLayerParams, FCParams
from src.models.nn import ActivationParams, NNTrainParams


def get_LeNet5_params(
    DatasetClass: type[CnnDataset],
    conv_compression: NNParamsCompMode = NNParamsCompMode.NONE,
    conv_bitwidth: int = 8,
    conv_activation: ActivationParams = ActivationParams(Activation.NONE),
    fc_compression: NNParamsCompMode = NNParamsCompMode.NONE,
    fc_bitwidth: int = 8,
    fc_activation: ActivationParams = ActivationParams(Activation.NONE),
    epochs: int = 1,
    early_stop_patience: int = 10,
) -> CNNParams:
    cnn_train_loader, cnn_test_loader = DatasetClass.get_dataloaders(50)

    conv_params = ConvParams(
        in_channels=DatasetClass.input_channels,
        in_dimensions=DatasetClass.input_dimensions,
        in_bitwidth=8,
        out_height=DatasetClass.output_size,
        layers=[
            ConvLayerParams(
                channels=32,
                kernel_size=5,
                stride=1,
                pooling_kernel_size=2,
                compression=conv_compression,
                bitwidth=conv_bitwidth,
            ),
            ConvLayerParams(
                channels=64,
                kernel_size=5,
                stride=1,
                pooling_kernel_size=2,
                compression=conv_compression,
                bitwidth=conv_bitwidth,
            ),
        ],
        reste_threshold=1.5,
        reste_o=3,
        activation=conv_activation,
        dropout_rate=0.1,
    )
    cnn_fc_params = FCParams(
        layers=[
            FCLayerParams(512, fc_compression, bitwidth=fc_bitwidth),
            FCLayerParams(
                DatasetClass.output_size, fc_compression, bitwidth=fc_bitwidth
            ),
        ],
        activation=fc_activation,
        qmode=QMode.DET,
        dropout_rate=0.0,
    )
    cnn_train_params = NNTrainParams(
        DatasetClass,
        cnn_train_loader,
        cnn_test_loader,
        epochs=epochs,
        learning_rate=0.001,
        weight_decay=0.00001,
        early_stop_patience=early_stop_patience,
    )
    cnn_params = CNNParams(
        in_bitwidth=8,
        conv=conv_params,
        fc=cnn_fc_params,
        train=cnn_train_params,
    )
    return cnn_params

In [3]:
params = get_LeNet5_params(
            DatasetClass=MiniMNISTDataset,
            conv_compression=NNParamsCompMode.NONE,
            conv_activation=ActivationParams(Activation.NONE),
            fc_compression=NNParamsCompMode.NONE,
            fc_activation=ActivationParams(Activation.NONE),
        )
model = params.get_model()
model.inspect_conv_layers()

INFO: Inspecting convolutional layers...
INFO: Input shape: torch.Size([1, 1, 28, 28]), equating to 784 inputs
INFO: Next layer shape: torch.Size([1, 32, 12, 12]), equating to 4608 inputs
INFO: Next layer shape: torch.Size([1, 64, 4, 4]), equating to 1024 inputs
INFO: FC input size is 1024


In [ ]:
from src.datasets.mnist_dataset import MiniMNISTDataset


def evaluate_compression_on_LeNet5(
    DatasetCls: type[CnnDataset],
    conv_compression: NNParamsCompMode,
    conv_bitwidth: int,
    conv_activation: Activation,
    fc_compression: NNParamsCompMode,
    fc_bitwidth: int,
    fc_activation: Activation,
) -> dict | None:
    print(f"Evaluating LeNet5 with {conv_compression} and {conv_activation}...")

    model_params = get_LeNet5_params(
        DatasetClass=DatasetCls,
        conv_compression=conv_compression,
        conv_bitwidth=conv_bitwidth,
        conv_activation=ActivationParams(conv_activation),
        fc_compression=fc_compression,
        fc_bitwidth=fc_bitwidth,
        fc_activation=ActivationParams(fc_activation),
    )
    evaluator = NNArchitectureEvaluator(model_params.train)

    try:
        stats = evaluator.evaluate_accuracy(model_params, times=1)
    except Exception as e:
        logging.error(
            f"Error evaluating {conv_compression} with {conv_activation}: {e}"
        )
        return None

    return {
        # Properties
        "architecture": "LeNet5",
        "dataset": MiniMNISTDataset.__name__,
        "conv_compression": conv_compression.name,
        "conv_activation": conv_activation.name,
        "fc_compression": fc_compression.name,
        "fc_activation": fc_activation.name,
        # Stats
        "best": stats["max"],
        "mean": stats["mean"],
        "accuracies": stats["accuracies"],
    }


datapoints = []


for activation in Activation:
    for conv_compression in NNParamsCompMode:
        if conv_compression == NNParamsCompMode.TERNARY:
            continue

        full = evaluate_compression_on_LeNet5(
            MiniMNISTDataset,
            conv_compression,
            8,
            activation,
            conv_compression,
            8,
            activation,
        )
        if full is not None:
            datapoints.append(full)

        # Without compressing the FC layers
        partial = evaluate_compression_on_LeNet5(
            MiniMNISTDataset,
            conv_compression,
            8,
            activation,
            NNParamsCompMode.NONE,
            8,
            Activation.RELU,
        )
        if partial is not None:
            datapoints.append(partial)

Evaluating LeNet5 with NNParamsCompMode.BINARY and Activation.NONE...
Evaluating LeNet5 with NNParamsCompMode.BINARY and Activation.NONE...
Evaluating LeNet5 with NNParamsCompMode.NBITS and Activation.NONE...
Evaluating LeNet5 with NNParamsCompMode.NBITS and Activation.NONE...
Evaluating LeNet5 with NNParamsCompMode.NONE and Activation.NONE...
Evaluating LeNet5 with NNParamsCompMode.NONE and Activation.NONE...
Evaluating LeNet5 with NNParamsCompMode.BINARY and Activation.BINARIZE...
Evaluating LeNet5 with NNParamsCompMode.BINARY and Activation.BINARIZE...


KeyboardInterrupt: 

In [30]:
datapoints[:2]

[{'architecture': 'LeNet5',
  'dataset': 'MiniMNISTDataset',
  'conv_compression': <NNParamsCompMode.BINARY: 'binary'>,
  'conv_activation': <Activation.NONE: 'none'>,
  'fc_compression': <NNParamsCompMode.BINARY: 'binary'>,
  'fc_activation': <Activation.NONE: 'none'>,
  'top-1': 94.25,
  'mean': np.float64(92.9),
  'accuracies': [93.5, 92.875, 91.875, 92.0, 94.25]},
 {'architecture': 'LeNet5',
  'dataset': 'MiniMNISTDataset',
  'conv_compression': <NNParamsCompMode.BINARY: 'binary'>,
  'conv_activation': <Activation.NONE: 'none'>,
  'fc_compression': <NNParamsCompMode.NONE: 'none'>,
  'fc_activation': <Activation.RELU: 'relu'>,
  'top-1': 94.875,
  'mean': np.float64(93.925),
  'accuracies': [94.875, 93.0, 94.875, 94.375, 92.5]}]

In [ ]:
import pandas as pd

df = pd.DataFrame(datapoints)
df

,architecture,dataset,conv_compression,conv_activation,fc_compression,fc_activation,top-1,mean,accuracies
0,LeNet5,MiniMNISTDataset,BINARY,NONE,BINARY,NONE,94.250,92.900,"[93.5, 92.875, 91.875, 92.0, 94.25]"
1,LeNet5,MiniMNISTDataset,BINARY,NONE,NONE,RELU,94.875,93.925,"[94.875, 93.0, 94.875, 94.375, 92.5]"
2,LeNet5,MiniMNISTDataset,NBITS,NONE,NBITS,NONE,93.375,91.425,"[90.25, 91.625, 92.75, 89.125, 93.375]"
3,LeNet5,MiniMNISTDataset,NBITS,NONE,NONE,RELU,93.500,91.000,"[91.75, 86.625, 93.5, 91.375, 91.75]"
4,LeNet5,MiniMNISTDataset,NONE,NONE,NONE,NONE,94.000,93.125,"[94.0, 91.625, 93.25, 92.875, 93.875]"
5,LeNet5,MiniMNISTDataset,NONE,NONE,NONE,RELU,96.000,95.350,"[96.0, 94.25, 95.75, 94.875, 95.875]"
6,LeNet5,MiniMNISTDataset,BINARY,BINARIZE,BINARY,BINARIZE,90.875,89.875,"[90.875, 89.875, 90.125, 89.0, 89.5]"


In [38]:
import plotly.express as px


fig = px.parallel_categories(
    df,
    dimensions=[
        "conv_activation",
        "conv_compression",
        "fc_compression",
        "mean"
    ],
    color_continuous_scale=px.colors.sequential.Inferno,
)
fig.show()

In [ ]:


fig = px.parallel_categories(
    df,
    dimensions=[
        "conv_activation",
        "conv_compression",
        "fc_compression",
        "best"
    ],
    color_continuous_scale=px.colors.sequential.Inferno,
)
fig.show()